In [1]:
from graphql.utils.schema_printer import print_schema
from graphql_compiler.macros import (
    register_macro_edge, get_schema_with_macros, perform_macro_expansion
)

from demo.server.demo_helpers import (
    get_schema, execute_query, pretty_print_data, get_empty_macro_registry, set_verbose_mode
)

In [2]:
macro_registry = get_empty_macro_registry()

## Macro edges
Edges that are "real" in the schema, but are actually defined using GraphQL syntax and don't really exist in the underlying database(s).

They are especially valuable for:
- hiding database normalization
- ensuring that different services use consistent definitions and methodologies
- defining domain-specific functionality for a specific product

This is how you define a new macro edge, called `out_Region_RegisteredAirlines` between `Region` and `Airline`.
It accounts for the fact that airlines are registered in *countries*, and not *geographic regions*, then hides that from the user.

In [3]:
macro_edge_graphql = '''{
    Region @macro_edge_definition(name: "out_Region_RegisteredAirlines") {
        out_GeographicArea_SubArea @recurse(depth: 5) {
            ... on Country {
                in_Airline_RegisteredIn @macro_edge_target {
                    id
                }
            }
        }
    }
}'''
macro_edge_args = {}
register_macro_edge(macro_registry, macro_edge_graphql, macro_edge_args)

Similarly, this is how we define a new macro that allows us to easily look up all airports in a region.

In [4]:
macro_edge_graphql = '''{
    Region @macro_edge_definition(name: "out_Region_ContainedAirports") {
        out_GeographicArea_SubArea @recurse(depth: 5) {
            ... on Country {
                in_Airport_BasedIn @macro_edge_target {
                    id
                }
            }
        }
    }
}'''
macro_edge_args = {}
register_macro_edge(macro_registry, macro_edge_graphql, macro_edge_args)

Defined macros become part of the schema, and are invisible to the querying user.

In [5]:
schema_with_macros = get_schema_with_macros(macro_registry)
print(print_schema(schema_with_macros))

schema {
  query: RootSchemaQuery
}

directive @filter(op_name: String!, value: [String!]!) on FIELD | INLINE_FRAGMENT

directive @tag(tag_name: String!) on FIELD

directive @output(out_name: String!) on FIELD

directive @output_source on FIELD

directive @optional on FIELD

directive @recurse(depth: Int!) on FIELD

directive @fold on FIELD

type Airline {
  _x_count: Int
  alpha2_country: String
  callsign: String
  iata_code: String
  icao_code: String
  id: Int
  in_FlightRoute_OperatingAirline: [FlightRoute]
  name: String
  out_Airline_RegisteredIn: [Country]
}

type Airport {
  _x_count: Int
  alpha2_country: String
  city_served: String
  elevation_ft: Int
  iata_code: String
  icao_code: String
  id: Int
  in_FlightRoute_FromAirport: [FlightRoute]
  in_FlightRoute_ToAirport: [FlightRoute]
  name: String
  out_Airport_BasedIn: [Country]
}

type Country implements GeographicArea {
  _x_count: Int
  alpha2: String
  alpha3: String
  in_GeographicArea_SubArea: [GeographicArea]
  na

When a query uses a macro edge, the macro is expanded before execution – producing a valid, macro-less query.

In [6]:
# European airlines with names starting with "W" and ending in "Air"
query = '''{
    Region {
        name @filter(op_name: "=", value: ["$region"])
        
        out_Region_RegisteredAirlines {
            name @filter(op_name: "ends_with", value: ["$suffix"])
                 @filter(op_name: "starts_with", value: ["$prefix"])
                 @output(out_name: "airline")
        }
    }
}'''
args = {
    'region': 'Europe',
    'prefix': 'W',
    'suffix': 'Air',
}
expanded_query, expanded_args = perform_macro_expansion(macro_registry, query, args)
print(expanded_query)
print(expanded_args)
_, result = execute_query(expanded_query, expanded_args)
pretty_print_data(result)

{
  Region {
    name @filter(op_name: "=", value: ["$region"])
    out_GeographicArea_SubArea @recurse(depth: 5) {
      ... on Country {
        in_Airline_RegisteredIn {
          name @filter(op_name: "ends_with", value: ["$suffix"]) @filter(op_name: "starts_with", value: ["$prefix"]) @output(out_name: "airline")
        }
      }
    }
  }
}

{'region': 'Europe', 'prefix': 'W', 'suffix': 'Air'}


,airline
0,Welcome Air
1,Windrose Air
2,Wizz Air


### Macros with arguments

Macro edges can take arguments as well. Consider the below macro, that connects `Airport` vertices to all other `Airport` vertices which can be reached by taking only direct flights operated by European carriers.

In [7]:
macro_edge_graphql = '''{
    Airport @macro_edge_definition(name: "out_DirectlyConnectedViaEuropeanAirline_ToAirport") {
        in_FlightRoute_FromAirport {
            stops @filter(op_name: "=", value: ["$stops"])
        
            out_FlightRoute_OperatingAirline {
                out_Airline_RegisteredIn {
                    in_GeographicArea_SubArea @recurse(depth: 5) {
                        name @filter(op_name: "=", value: ["$region_name"])
                    }
                }
            }
            out_FlightRoute_ToAirport @macro_edge_target {
                id
            }
        }
    }
}'''
macro_edge_args = {
    'region_name': 'Europe',
    'stops': 0,
}
register_macro_edge(macro_registry, macro_edge_graphql, macro_edge_args)

In [8]:
# Direct flights operated by European airlines starting in the Caribbean
query = '''{
    Region {
        name @filter(op_name: "=", value: ["$region"])
        
        out_Region_ContainedAirports {
            name @output(out_name: "airport_name")
            city_served @output(out_name: "airport_city")
            iata_code @output(out_name: "airport_code")
            
            out_DirectlyConnectedViaEuropeanAirline_ToAirport {
                name @output(out_name: "destination_name")
                city_served @output(out_name: "destination_city")
                iata_code @output(out_name: "destination_code")
            }
        }
    }
}'''
args = {
    'region': 'Caribbean',
}
expanded_query, expanded_args = perform_macro_expansion(macro_registry, query, args)
print(expanded_query)
print(expanded_args)
_, result = execute_query(expanded_query, expanded_args)
pretty_print_data(result)

{
  Region {
    name @filter(op_name: "=", value: ["$region"])
    out_GeographicArea_SubArea @recurse(depth: 5) {
      ... on Country {
        in_Airport_BasedIn {
          name @output(out_name: "airport_name")
          city_served @output(out_name: "airport_city")
          iata_code @output(out_name: "airport_code")
          in_FlightRoute_FromAirport {
            stops @filter(op_name: "=", value: ["$stops"])
            out_FlightRoute_OperatingAirline {
              out_Airline_RegisteredIn {
                in_GeographicArea_SubArea @recurse(depth: 5) {
                  name @filter(op_name: "=", value: ["$region_name"])
                }
              }
            }
            out_FlightRoute_ToAirport {
              name @output(out_name: "destination_name")
              city_served @output(out_name: "destination_city")
              iata_code @output(out_name: "destination_code")
            }
          }
        }
      }
    }
  }
}

{'region': 'Caribbean', 'r

,airport_city,airport_code,airport_name,destination_city,destination_code,destination_name
0,The Valley,AXA,Clayton J Lloyd International Airport,Philipsburg,SXM,Princess Juliana International Airport
1,Antigua,ANU,V.C. Bird International Airport,Charlestown,NEV,Vance W. Amory International Airport
2,Antigua,ANU,V.C. Bird International Airport,London,LGW,London Gatwick Airport
3,Antigua,ANU,V.C. Bird International Airport,Punta Cana,PUJ,Punta Cana International Airport
4,Antigua,ANU,V.C. Bird International Airport,Basse Terre,SKB,Robert L. Bradshaw International Airport
5,Antigua,ANU,V.C. Bird International Airport,Scarborough,TAB,Tobago-Crown Point Airport
6,Antigua,ANU,V.C. Bird International Airport,Frankfurt,FRA,Frankfurt am Main Airport
7,Antigua,ANU,V.C. Bird International Airport,London,LGW,London Gatwick Airport
8,Antigua,ANU,V.C. Bird International Airport,Punta Cana,PUJ,Punta Cana International Airport
9,Antigua,ANU,V.C. Bird International Airport,Basse Terre,SKB,Robert L. Bradshaw International Airport


# I promised cross-database querying. So where is it?

We've been doing it all along! The countries and regions data is in a graph database called OrientDB, and the airlines, airports, and flights information is in Postgres.

You probably have noticed that `execute_query` always returned two arguments, and we always ignored the first one. That's the distributed query plan!

In [9]:
from graphql_compiler.schema_transformation.query_plan import print_query_plan

In [10]:
query_plan, result = execute_query(expanded_query, expanded_args)
print(print_query_plan(query_plan))


Execute subplan ID 0 in schema named "orientdb":
{
  Region {
    name @filter(op_name: "=", value: ["$region"])
    out_GeographicArea_SubArea @recurse(depth: 5) {
      ... on Country {
        alpha2 @output(out_name: "__intermediate_output_0")
      }
    }
  }
}

    Execute subplan ID 1 in schema named "postgres":
    {
      Airport {
        name @output(out_name: "airport_name")
        city_served @output(out_name: "airport_city")
        iata_code @output(out_name: "airport_code")
        alpha2_country @output(out_name: "__intermediate_output_1") @filter(op_name: "in_collection", value: ["$__intermediate_output_0"])
        in_FlightRoute_FromAirport {
          stops @filter(op_name: "=", value: ["$stops"])
          out_FlightRoute_OperatingAirline {
            alpha2_country @output(out_name: "__intermediate_output_2")
          }
          out_FlightRoute_ToAirport {
            name @output(out_name: "destination_name")
            city_served @output(out_name: "dest

In [11]:
set_verbose_mode(True)
query_plan, result = execute_query(expanded_query, expanded_args)
set_verbose_mode(False)


Executing OrientDB MATCH against OrientDB:
 SELECT Region__out_GeographicArea_SubArea___1.alpha2 AS `__intermediate_output_0` FROM  ( MATCH  { class: Region, where: ((name = "Caribbean")), as: Region___1 }.out('GeographicArea_SubArea') { while: ($depth < 5), where: ((@this INSTANCEOF 'Country')), as: Region__out_GeographicArea_SubArea___1 } RETURN $matches)

Executing SQL against Postgres:
 SELECT "Airport_1".alpha2_country AS __intermediate_output_1, "Airline_1".alpha2_country AS __intermediate_output_2, "Airport_1".city_served AS airport_city, "Airport_1".iata_code AS airport_code, "Airport_1".name AS airport_name, "Airport_2".city_served AS destination_city, "Airport_2".iata_code AS destination_code, "Airport_2".name AS destination_name 
FROM "Airport" AS "Airport_1" JOIN "FlightRoute" AS "FlightRoute_1" ON "Airport_1".id = "FlightRoute_1".source_airport_id JOIN "Airline" AS "Airline_1" ON "FlightRoute_1".airline_id = "Airline_1".id JOIN "Airport" AS "Airport_2" ON "FlightRoute_1".

In [ ]:
from graphql_compiler.schema_transformation.merge_schemas import (
    CrossSchemaEdgeDescriptor, FieldReference,
)
cross_schema_edges = [
    CrossSchemaEdgeDescriptor(
        edge_name='Airport_BasedIn',
        outbound_field_reference=FieldReference(
            schema_id='postgres',
            type_name='Airport',
            field_name='alpha2_country',
        ),
        inbound_field_reference=FieldReference(
            schema_id='orientdb',
            type_name='Country',
            field_name='alpha2',
        ),
        out_edge_only=False,
    ),
    CrossSchemaEdgeDescriptor(
        edge_name='Airline_RegisteredIn',
        outbound_field_reference=FieldReference(
            schema_id='postgres',
            type_name='Airline',
            field_name='alpha2_country',
        ),
        inbound_field_reference=FieldReference(
            schema_id='orientdb',
            type_name='Country',
            field_name='alpha2',
        ),
        out_edge_only=False,
    )
]